In [1]:
import sys

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [2]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window
 
def input_event_paths(date, depth):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"/user/inthemidde/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in range(depth)]

def get_spark():
    conf = SparkConf().setAppName(f"EventsPartitioningJob_i028159")
    sc = SparkContext(conf=conf)
    return SQLContext(sc)

def partition_writer(events):
    return events \
        .write \
        .mode('overwrite') \
        .partitionBy('event_type') \
        .format('parquet')

def prepare_parquet(sql, date):
    base_input_path = '/user/master/data/events'
    base_output_path = 'hdfs://rc1a-dataproc-m-dg5lgqqm7jju58f9.mdb.yandexcloud.net/user/inthemidde/data/events'
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    for i in range(0,100,1): #range(start, stop[, step]):  
# данные, ранее 2022-01-15 и позднее 2022-06-01 (вкл) содержат не все поля схемы (напр, нет поле message_channel_from)
# , поэтому пересчет в будущем вызывает ошибки runtime
# Если сохранили данные в файлах с неполной схемой, то такие файлы лучше удалить
# hdfs dfs -rm -R /user/inthemidde/data/events/date=2022-01-14
        lv_date = (dt-datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        print('lv_date=', lv_date)
        events = sql.read.json(f"{base_input_path}/date={lv_date}")
        writer = partition_writer(events)
        writer.save(f'{base_output_path}/date={lv_date}')
    print('All done!!')


In [3]:
def reaction_tag_tops(date, depth, spark):
    reaction_paths = input_event_paths(date, depth)
    print(f'reaction_paths={reaction_paths}')
    reactions = spark.read\
        .option("basePath", "/user/inthemidde/data/events")\
        .parquet(*reaction_paths)\
        .where("event_type='reaction'")
    
    all_message_tags = spark.read.parquet("/user/inthemidde/data/events")\
        .where("event_type='message' and event.message_channel_to is not null")\
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag")
        )
 
    reaction_tags = reactions\
        .select(F.col("event.reaction_from").alias("user_id"), 
                F.col("event.message_id").alias("message_id"), 
                F.col("event.reaction_type").alias("reaction_type")
        ).join(all_message_tags.select("message_id", "tag"), "message_id")
 
    reaction_tops = reaction_tags\
        .groupBy("user_id", "tag", "reaction_type")\
        .agg(F.count("*").alias("tag_count"))\
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id", "reaction_type")\
        .orderBy(F.desc("tag_count"), F.desc("tag"))))\
        .where("rank <= 3")\
        .groupBy("user_id", "reaction_type")\
        .pivot("rank", [1, 2, 3])\
        .agg(F.first("tag"))\
        .cache()
 
    like_tops = reaction_tops\
        .where("reaction_type = 'like'")\
        .drop("reaction_type")\
        .withColumnRenamed("1", "like_tag_top_1")\
        .withColumnRenamed("2", "like_tag_top_2")\
        .withColumnRenamed("3", "like_tag_top_3")
 
    dislike_tops = reaction_tops\
        .where("reaction_type = 'dislike'")\
        .drop("reaction_type")\
        .withColumnRenamed("1", "dislike_tag_top_1")\
        .withColumnRenamed("2", "dislike_tag_top_2")\
        .withColumnRenamed("3", "dislike_tag_top_3")
 
    result = like_tops\
        .join(dislike_tops, "user_id", "full_outer")
    print(f'results={result}')
    return result

In [4]:
sql = get_spark()
print(f'spark={sql}')
# prepare_parquet(sql, '2022-06-04')
# prepare_parquet(sql, '2022-05-04')
# prepare_parquet(sql, '2022-06-04')

23/08/16 09:33:41 WARN Utils: Your hostname, fhm0o00hgvnb197n9pne resolves to a loopback address: 127.0.1.1; using 172.16.0.4 instead (on interface eth0)
23/08/16 09:33:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/16 09:33:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark=<pyspark.sql.context.SQLContext object at 0x7faedd795d90>


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [27]:
r = reaction_tag_tops('2022-05-04', 5, sql)
r.repartition(1).write.mode('overwrite').parquet('/user/inthemidde/data/tmp/reaction_tag_tops_05_04_5')
r = reaction_tag_tops('2022-04-04', 5, sql)
r.repartition(1).write.mode('overwrite').parquet('/user/inthemidde/data/tmp/reaction_tag_tops_04_04_5')
r = reaction_tag_tops('2022-04-04', 1, sql)
r.repartition(1).write.mode('overwrite').parquet('/user/inthemidde/data/tmp/reaction_tag_tops_04_04_1')


reaction_paths=['/user/inthemidde/data/events/date=2022-04-04']


results=DataFrame[user_id: string, like_tag_top_1: string, like_tag_top_2: string, like_tag_top_3: string, dislike_tag_top_1: string, dislike_tag_top_2: string, dislike_tag_top_3: string]


In [23]:
base_input_path = '/user/master/data/events'
# eventsbad = sql.read.json(f"{base_input_path}/date=2022-01-15")
eventsbad = sql.read.json(f"{base_input_path}/date=2022-01-14")
eventsbad.printSchema()
# events5 = sql.read.json(f"{base_input_path}/date=2022-05-31")
# events5.printSchema()

# print(f'events.columns={events.columns}')
# print(f'events={events}')

# all_message_tags = sql.read.parquet("/user/inthemidde/data/events/date=2022-06-05")
# print(all_message_tags.columns)
#     .where("event_type='message' and event.message_channel_to is not null")\
#     .select(F.col("event.message_id").alias("message_id"),
#             F.col("event.message_from").alias("user_id"),
#             F.explode(F.col("event.tags")).alias("tag")
#     )

root
 |-- event: struct (nullable = true)
 |    |-- datetime: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- message_from: long (nullable = true)
 |    |-- message_group: long (nullable = true)
 |    |-- message_id: long (nullable = true)
 |    |-- message_to: long (nullable = true)
 |    |-- message_ts: string (nullable = true)
 |    |-- subscription_user: string (nullable = true)
 |    |-- user: string (nullable = true)
 |-- event_type: string (nullable = true)

